<div class="alert alert-block alert-info">

# **LoadComicData.ipynb**
***
### **This script loads comic book images for neural network training. The images are made in a pair of one colored, and one grayscaled.**


In [3]:
import os
import zipfile
import tarfile
import glob
import cv2
import numpy as np
from tqdm import tqdm

In [4]:
os.environ['KAGGLE_CONFIG_DIR'] = 'CNN_U-Net_GAN_Project\kaggle.json'

In [10]:
print("Downloading dataset from Kaggle...")
!kaggle datasets download -d cenkbircanoglu/comic-books-classification -p comic_dataset --unzip

Dataset URL: https://www.kaggle.com/datasets/cenkbircanoglu/comic-books-classification
License(s): GPL-2.0




  0%|          | 0.00/2.27G [00:00<?, ?B/s]
  7%|▋         | 158M/2.27G [00:00<00:01, 1.66GB/s]
 14%|█▍        | 328M/2.27G [00:00<00:01, 1.72GB/s]
 21%|██▏       | 499M/2.27G [00:00<00:01, 1.75GB/s]
 29%|██▊       | 666M/2.27G [00:00<00:01, 1.74GB/s]
 36%|███▌      | 833M/2.27G [00:00<00:00, 1.74GB/s]
 43%|████▎     | 0.98G/2.27G [00:00<00:00, 1.76GB/s]
 51%|█████     | 1.15G/2.27G [00:00<00:00, 1.78GB/s]
 58%|█████▊    | 1.32G/2.27G [00:01<00:01, 634MB/s] 
 65%|██████▍   | 1.47G/2.27G [00:01<00:01, 773MB/s]
 72%|███████▏  | 1.64G/2.27G [00:01<00:00, 944MB/s]
 80%|███████▉  | 1.81G/2.27G [00:01<00:00, 1.11GB/s]
 87%|████████▋ | 1.97G/2.27G [00:01<00:00, 1.25GB/s]
 94%|█████████▍| 2.14G/2.27G [00:01<00:00, 1.38GB/s]
100%|██████████| 2.27G/2.27G [00:03<00:00, 717MB/s] 


### Extract the ZIP file

In [ ]:
def extract_dataset(zip_path, extract_to):
    print("Unzipping dataset...")
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_to)

### Extract the TAR.GZ inside the unzipped folder

In [ ]:
def extract_tar_gz(tar_gz_path, extract_to):
    print("Extracting .tar.gz archive...")
    with tarfile.open(tar_gz_path, 'r:gz') as tar_ref:
        tar_ref.extractall(extract_to)

### Load and process images